In [3]:
import polars as pl

auto_mapping_strings = [
    ("Verlengde QT",4008859,"Prolonged QT interval"),
    ("sinusaritmie", 4141820,"ECG: sinus arrhythmia"),
    ("premature ventriculaire complexen",4088985,"Multiple premature ventricular complexes"),
    ("premature ventricular complexes",4088985,"Multiple premature ventricular complexes"),
    ("premature ventricular complex",4089462,"Ventricular premature complex"),
    (r"ST .+T.+ T-golf afwijkingen",4327859,"Nonspecific ST-T abnormality on electrocardiogram"),
    ("Naar rechts gerichte as",4166218,"Right axis deviation"),
    ("premature ventriculaire of aberrant geleide complexen",4091901,"Aberrant premature complexes"),
    ("Ventricular-paced rhythm",4189347,"Ventricular rhythm from artificial pacing"),
    ("mogelijk digitaliseffect",4091454,"Digitalis ECG pattern"),
    ("\*\* \*\* ACUTE MI / STEMI \*\* \*\*",4064609,"EKG: myocardial infarction"),
    ("Atrial-sensed ventricular-paced rhythm",4304202,"Rhythm from artificial pacing"),
    ("Atrial-paced rhythm",4274333,"Atrial rhythm from artificial pacing"),
    ("prolonged AV conduction",314379,"First degree atrioventricular block"),
    ("1e graads AV-blok",314379,"First degree atrioventricular block"),
    ("AV dual-paced rhythm",4304202,"Rhythm from artificial pacing"),
    ("premature atriale complexen",4065285,"EKG: atrial ectopics"),
    ("junctionele pacemaker",4140001,"ECG: junctional rhythm"),
    ("kort PR",4171521,"Shortened PR interval"),
    ("ventricular-paced complexes",4066289,"Ventricular premature beats"),
    ("en fusiecomplexen",4029303,"Fusion beats"),
    ("AV-dissociatie",4175473,"Atrioventricular dissociation"),
    ("Versneld Junctioneel ritme",4145359,"EKG: accelerated junctional rhythm"),
    ("met 2:1 AV-geleiding",4065290,"ECG: partial atrioventricular block - 2:1"),
    ("Breed QRS-ritme",4321877,"Wide QRS complex"),
    ("ST depression",4095285,"ST segment depression"),
    ("Idioventriculair ritme",4171193,"Idioventricular rhythm"),
    ("Junctioneel ritme",4140001,"ECG: junctional rhythm"),
    ("premature supraventriculaire complexen",441872,"Supraventricular premature beats"),
    ("supraventricular complexes",441872,"Supraventricular premature beats"),
    ("met 3:1 AV-geleiding",4064458,"ECG: partial atrioventricular block - 3:1"),
    ("geblokkeerd premature atriale complexen",4088984,"Blocked premature atrial contraction"),
    ("ventriculaire escape complexen",4088507,"Ventricular escape complex"),
    ("slechte kwaliteit",4088345,"Poor ECG quality"),
    ("Slechte gegevenskwaliteit",4088345,"Poor ECG quality"),
    ("Fout",4088345,"Poor ECG quality"),
    ("junctionele tachycardie",4142573,"ECG: junctional tachycardia"),
]

data = pl.read_csv("./ecg_usagi_raw.csv")
for (string,code,name) in auto_mapping_strings:
    mapped_df = data.filter(pl.col("sourceName").str.to_lowercase().str.contains(rf"{string.lower()}")).clone()
    mapped_df = mapped_df.with_columns([
        pl.lit(code).cast(pl.Int64).alias("conceptId"), 
        pl.lit(name).alias("conceptName")
    ])
    data = data.vstack(mapped_df).unique()
data = data.sort(["sourceFrequency", "sourceName"], reverse=[True,False])
data.write_csv("./ecg_usagi_auto.csv")